In [ ]:
# !wget https://raw.githubusercontent.com/alexmascension/revisit_reynolds_fb/master/requirements.txt
# !pip install -r requirements.txt

In [ ]:
import scanpy as sc
import scanpy.external as sce

import pandas as pd
import numpy as np

import os

import triku as tk

import matplotlib.pyplot as plt
import matplotlib as mpl

from functools import reduce

In [ ]:
# To print versions of imports 

import types

def imports():
    for name, val in globals().items():
        if isinstance(val, types.ModuleType):
            yield val.__name__

excludes = ['builtins', 'types', 'sys']

imported_modules = [module for module in imports() if module not in excludes]

clean_modules = []

for module in imported_modules:

    sep = '.'  # to handle 'matplotlib.pyplot' cases
    rest = module.split(sep, 1)[0]
    clean_modules.append(rest)

changed_imported_modules = list(set(clean_modules))  # drop duplicates

pip_modules = !pip freeze  # you could also use `!conda list` with anaconda

for module in pip_modules:
    try:
        name, version = module.split('==')
        if name in changed_imported_modules:
            print(name + '\t' + version)
    except:
        pass

In [ ]:
seed = 0

In [ ]:
# Palettes for UMAP gene expression

magma = [plt.get_cmap('magma')(i) for i in np.linspace(0,1, 80)]
magma[0] = (0.88, 0.88, 0.88, 1)
magma = mpl.colors.LinearSegmentedColormap.from_list("", magma[:65])

In [ ]:
mpl.rcParams['figure.dpi'] = 150

In [ ]:
def assign_cats(adata, dict_cats, column_groupby='leiden', quantile_gene_sel=0.7, do_return=False, intermediate_states=False, diff=0.05, 
                key_added='assigned_cats'):
    """
    This functions uses a set of genes assigned to different categories so that leiden clusters can be assigned to one of these categories.
    For example, to categorize fibroblasts from pericytes, endothelial cells, or cells with high mitochondrial content.
    It could be done with each cell individually, but it is better to use clusters to discern the different categories because
    the method, although efficient, can sometimes be noisy due to the noisiness of the sc datasets.
    """
    
    for cat in list(dict_cats.keys()):
        mat_cat = np.zeros((len(adata), len(dict_cat[cat])), dtype=float)
        
        for gene_idx, gene in enumerate(dict_cat[cat]):
            try:
                mat_cat[:, gene_idx] = np.asarray(np.dot(adata.obsp['connectivities'], adata[:, gene].X).todense()).ravel() / adata.uns['neighbors']['params']['n_neighbors']
                
                # We normalize the expression to the maximum (per 98) of expression ** 0.5 to account to some extent for the expression of the gene. 
                # In the end, a gene highly expressed must account for more than a gene with lower expression
                mat_cat[mat_cat[:, gene_idx] > 0, gene_idx] = np.argsort(np.argsort(mat_cat[mat_cat[:, gene_idx] > 0, gene_idx]))
                mat_cat[:, gene_idx] /= np.max(mat_cat[:, gene_idx])
            except:
                print(f'Gene {gene} is not on the list')    
            
        sum_mat_cat = np.asarray(mat_cat.mean(1)).ravel()       
        adata.obs[cat] = sum_mat_cat
    
    score_per_cluster = adata.obs[[column_groupby] + list(dict_cats.keys())].groupby(column_groupby).quantile(quantile_gene_sel)
    max_cat_dict_std = dict(zip(score_per_cluster.std(1).index, score_per_cluster.std(1).values))
    adata.obs[f'{key_added}_std'] = [max_cat_dict_std[i] for i in adata.obs[column_groupby]]
    max_cat_dict_mean = dict(zip(score_per_cluster.mean(1).index, score_per_cluster.mean(1).values))
    adata.obs[f'{key_added}_mean'] = [max_cat_dict_mean[i] for i in adata.obs[column_groupby]]
    adata.obs[f'{key_added}_CV'] = adata.obs[f'{key_added}_mean'] / adata.obs[f'{key_added}_std']
    
    for cat in score_per_cluster.columns:
        max_cat_dict = dict(zip(score_per_cluster.index, score_per_cluster[cat].values))
        adata.obs[f'{key_added}_{cat}'] = [max_cat_dict[i] for i in adata.obs[column_groupby]]
    
    if intermediate_states: # For each cluster we will identify which categories are close to the highest one, and merge their names.
        list_names_cats_per_cluster = []
        for cluster in score_per_cluster.index:
            scores_cluster = score_per_cluster.loc[cluster]
            scores_cluster = scores_cluster[scores_cluster > scores_cluster.max() - diff]
            list_names_cats_per_cluster.append('/'.join(scores_cluster.index))
            
        inter_cat_dict = dict(zip(score_per_cluster.idxmax(axis=1).index, list_names_cats_per_cluster))
        adata.obs[f'{key_added}'] = [str(inter_cat_dict[i]) for i in adata.obs[column_groupby]]
    else:
        max_cat_dict = dict(zip(score_per_cluster.idxmax(axis=1).index, score_per_cluster.idxmax(axis=1).values))
        adata.obs[f'{key_added}'] = [str(max_cat_dict[i]) for i in adata.obs[column_groupby]]
    
    if do_return:
        return score_per_cluster

# Reynolds et al. 2020 dataset download and preprocess
In this section we are going to download and process the anndata files for the 5 healthy samples (S1 to S5). We are first going to use the processed files from Reynolds et al., because they contain the main fb populations, which we are interested in (FB1-3). With that, we can partially replicate the analysis. However, the data richness is not that good (the UMAPs are more *blooby*) so we preprocess the FASTQ files on our own, and used the processed adatas.

Once we have our own anndatas, the preprocessing is the following:
* Assign FB types based on Reynolds adatas to our adatas. Some cells will be unassigned.
* QC metrics 
* Filter genes (min counts = 30)
* Set raw X
* Normalize and log1p
* PCA + triku + neighbors + umap + leiden
* Use  `assign_cats` to assign leiden clusters to cell populations with selected markers (markers for each dataset may vary!)
* Filter adata to retain only fibroblasts
* Filter genes to remove 0 counts
* PCA + triku + neighbors + umap + leiden [leiden is not used here but may be used later]
* Check if strange populations appear and filter them in `assign_cats`, then repeat the last steps.

In [ ]:
os.getcwd()

In [ ]:
reynolds_dir = 'reynolds_2020'
os.makedirs(reynolds_dir, exist_ok=True)

### Making and saving the fb healthy dataset to zenodo

In [ ]:
# adata_reynolds = sc.read('submission_210120.h5ad', backup_url='https://zenodo.org/record/4536165/files/submission_210120.h5ad')
# adata_reynolds_fb = adata_reynolds[(adata_reynolds.obs['full_clustering'].isin(['F1', 'F2', 'F3'])) & 
#                                    (adata_reynolds.obs['Status'] == 'Healthy')]
# sc.pp.filter_genes(adata_reynolds_fb, min_counts=100)
# del adata_reynolds_fb.var

# for obs in ['mad_prd', 'Status', 'Site', 'Tissue', 'Enrichment', 'Location', 'Sex', 'Age', 'stage']:
#     del adata_reynolds_fb.obs[obs]
    
# adata_reynolds_fb.write_h5ad(reynolds_dir + '/reynolds_2020_fb_healthy.h5ad')

### Direct h5ad download

In [ ]:
adata_reynolds_fb_healthy = sc.read(reynolds_dir + '/reynolds_2020_fb_healthy.h5ad', 
                                    backup_url='https://zenodo.org/record/4605340/files/reynolds_2020_fb_healthy.h5ad?download=1')

In [ ]:
sc.pp.filter_genes(adata_reynolds_fb_healthy, min_counts=50)

In [ ]:
sc.pp.log1p(adata_reynolds_fb_healthy)
sc.pp.normalize_total(adata_reynolds_fb_healthy)

In [ ]:
df_batches = pd.DataFrame(np.unique(adata_reynolds_fb_healthy.obs['sample_id'], return_counts=True)).transpose()

In [ ]:
df_batches.sort_values(by=1, ascending=False)

In [ ]:
selected_samples = df_batches[df_batches[1] > 50][0].values

In [ ]:
adata_reynolds_fb_healthy = adata_reynolds_fb_healthy[adata_reynolds_fb_healthy.obs['sample_id'].isin(selected_samples)]  #selected_samples)]

In [ ]:
adata_reynolds_fb_healthy

In [ ]:
sc.pp.filter_genes(adata_reynolds_fb_healthy, min_counts=1)

In [ ]:
sc.pp.pca(adata_reynolds_fb_healthy, random_state=seed, n_comps=30)
sce.pp.bbknn(adata_reynolds_fb_healthy, metric='angular', batch_key='sample_id')
tk.tl.triku(adata_reynolds_fb_healthy, n_procs=1, random_state=seed, use_adata_knn=True)

In [ ]:
sc.tl.umap(adata_reynolds_fb_healthy, min_dist=0.1, random_state=seed)

In [ ]:
sc.tl.leiden(adata_reynolds_fb_healthy, resolution=1.5, random_state=seed)

In [ ]:
sc.pl.umap(adata_reynolds_fb_healthy, color=['leiden', 'sample_id', 'full_clustering'], legend_loc='on data')

In [ ]:
sc.pl.umap(adata_reynolds_fb_healthy, color=['APCDD1', 'COL18A1', 'COMP', 'SLPI', 'WIF1'], cmap=magma, use_raw=False)

In [ ]:
sc.pl.umap(adata_reynolds_fb_healthy, color=['MT2A', 'CCL19', 'CCL2', 'CD46'], cmap=magma, use_raw=False)

In [ ]:
sc.pl.umap(adata_reynolds_fb_healthy, color=['POSTN', 'COMP', 'COCH'], cmap=magma, use_raw=False)

## 4820STDY7388991 (S1)

### Reynolds preprocessed

In [ ]:
adata_reynolds_fb_4820STDY7388991 = adata_reynolds_fb_healthy[adata_reynolds_fb_healthy.obs['sample_id'].isin(['4820STDY7388991'])]  #selected_samples)]

In [ ]:
adata_reynolds_fb_4820STDY7388991

In [ ]:
sc.pp.filter_genes(adata_reynolds_fb_4820STDY7388991, min_counts=1)
adata_reynolds_fb_4820STDY7388991 = adata_reynolds_fb_4820STDY7388991[:, adata_reynolds_fb_4820STDY7388991.var_names != 'C3']

In [ ]:
sc.pp.pca(adata_reynolds_fb_4820STDY7388991, random_state=seed, n_comps=30)
tk.tl.triku(adata_reynolds_fb_4820STDY7388991, n_procs=1, random_state=seed, use_adata_knn=True)
sc.pp.neighbors(adata_reynolds_fb_4820STDY7388991, metric='cosine', random_state=seed)

In [ ]:
sc.tl.umap(adata_reynolds_fb_4820STDY7388991, min_dist=0.3, random_state=seed)

In [ ]:
sc.tl.leiden(adata_reynolds_fb_4820STDY7388991, resolution=1, random_state=seed)

In [ ]:
sc.pl.umap(adata_reynolds_fb_4820STDY7388991, color=['leiden', 'sample_id', 'full_clustering'], legend_loc='on data')

In [ ]:
sc.pl.umap(adata_reynolds_fb_4820STDY7388991, color=['APCDD1', 'COL18A1', 'COMP', 'SLPI', 'WIF1'], cmap=magma, use_raw=False)

In [ ]:
sc.pl.umap(adata_reynolds_fb_4820STDY7388991, color=['MT2A', 'CCL19', 'CCL2', 'CD46'], cmap=magma, use_raw=False)

In [ ]:
sc.pl.umap(adata_reynolds_fb_4820STDY7388991, color=['POSTN', 'COMP', 'COCH'], cmap=magma, use_raw=False)

### Direct loom

In [ ]:
adata_reynolds_fb_4820STDY7388991_loom = sc.read('reynolds_2020/reynolds_2020_0_4820STDY7388991_s1_dermis_fibroblasts.loom', 
                                                 backup_url='https://zenodo.org/record/4596375/files/reynolds_2020_0_4820STDY7388991_s1_dermis_fibroblasts.loom')
adata_reynolds_fb_4820STDY7388991_loom.var_names_make_unique()

adata_reynolds_fb_4820STDY7388991_loom = adata_reynolds_fb_4820STDY7388991_loom[:, adata_reynolds_fb_4820STDY7388991_loom.var_names != 'C3']

In [ ]:
adata_reynolds_fb_4820STDY7388991_loom.obs_names = [f"{i.split('_')[-1]}-1-{i.split('_')[1]}" for i in adata_reynolds_fb_4820STDY7388991_loom.obs_names]

In [ ]:
df_fb_type = pd.Series('-', index=adata_reynolds_fb_4820STDY7388991_loom.obs_names)
df_fb_type.loc[adata_reynolds_fb_4820STDY7388991.obs_names & adata_reynolds_fb_4820STDY7388991_loom.obs_names] = adata_reynolds_fb_4820STDY7388991[adata_reynolds_fb_4820STDY7388991.obs_names & adata_reynolds_fb_4820STDY7388991_loom.obs_names].obs['full_clustering']
adata_reynolds_fb_4820STDY7388991_loom.obs['science_clustering'] = df_fb_type

In [ ]:
sc.pp.calculate_qc_metrics(adata_reynolds_fb_4820STDY7388991_loom, inplace=True)

In [ ]:
sc.pl.violin(adata_reynolds_fb_4820STDY7388991_loom, ['n_genes_by_counts'])
sc.pl.violin(adata_reynolds_fb_4820STDY7388991_loom, ['log1p_total_counts'])

In [ ]:
sc.pp.filter_genes(adata_reynolds_fb_4820STDY7388991_loom, min_counts=30)
adata_reynolds_fb_4820STDY7388991_loom.raw = adata_reynolds_fb_4820STDY7388991_loom

In [ ]:
sc.pp.normalize_per_cell(adata_reynolds_fb_4820STDY7388991_loom)
sc.pp.log1p(adata_reynolds_fb_4820STDY7388991_loom)

In [ ]:
sc.pp.pca(adata_reynolds_fb_4820STDY7388991_loom, random_state=seed, n_comps=30)
tk.tl.triku(adata_reynolds_fb_4820STDY7388991_loom, n_procs=1, random_state=seed, use_adata_knn=True)
sc.pp.neighbors(adata_reynolds_fb_4820STDY7388991_loom, metric='cosine', random_state=seed)

In [ ]:
sc.tl.umap(adata_reynolds_fb_4820STDY7388991_loom, min_dist=0.3, random_state=seed)
sc.tl.leiden(adata_reynolds_fb_4820STDY7388991_loom, resolution=3.2, random_state=seed)

In [ ]:
sc.pl.umap(adata_reynolds_fb_4820STDY7388991_loom, color=['leiden', 'science_clustering'], legend_loc='on data')

In [ ]:
sc.pl.umap(adata_reynolds_fb_4820STDY7388991_loom, color=['LUM', 'PDGFRA', 'VIM', 'DCN', 'COL1A1', 'SFRP2', 'APOE', 'POSTN'], legend_loc='on data', cmap=magma)

In [ ]:
sc.tl.rank_genes_groups(adata_reynolds_fb_4820STDY7388991_loom, groupby='leiden')
sc.pl.rank_genes_groups(adata_reynolds_fb_4820STDY7388991_loom)

In [ ]:
dict_cat = {'fb': ['LUM', 'PDGFRA', 'COL1A1'], 'peri': ['RGS5', 'MYL9', 'NDUFA4L2'], 'eritro': ['HBB', 'HBA2', 'HBA1'], 
            'immune': ['TPSB2', 'TPSAB1', 'HLA-DRA', 'FCER1G', 'CD74'], 'melanocyte': ['PMEL', 'MLANA'], 
            'mt': ['MTND2P28', 'MTND4P12', 'MTCO1P40', 'ADAM33', 'RN7SL2', 'MTRNR2L6']}

assign_cats(adata_reynolds_fb_4820STDY7388991_loom, dict_cat)

In [ ]:
sc.pl.umap(adata_reynolds_fb_4820STDY7388991_loom, color=['leiden', 'science_clustering', 'assigned_cats'], legend_loc='on data', cmap=magma)

In [ ]:
adata_reynolds_fb_4820STDY7388991_loom_fb = adata_reynolds_fb_4820STDY7388991_loom[adata_reynolds_fb_4820STDY7388991_loom.obs['assigned_cats'] == 'fb']

In [ ]:
sc.pp.filter_genes(adata_reynolds_fb_4820STDY7388991_loom_fb, min_counts=1)

In [ ]:
sc.pp.pca(adata_reynolds_fb_4820STDY7388991_loom_fb, random_state=seed, n_comps=30)
tk.tl.triku(adata_reynolds_fb_4820STDY7388991_loom_fb, n_procs=1, random_state=seed, use_adata_knn=True)
sc.pp.neighbors(adata_reynolds_fb_4820STDY7388991_loom_fb, metric='cosine', random_state=seed)

In [ ]:
sc.tl.umap(adata_reynolds_fb_4820STDY7388991_loom_fb, min_dist=0.2, random_state=seed)
sc.tl.leiden(adata_reynolds_fb_4820STDY7388991_loom_fb, resolution=1, random_state=seed)

In [ ]:
sc.pl.umap(adata_reynolds_fb_4820STDY7388991_loom_fb, color=['leiden', 'science_clustering'], cmap=magma, use_raw=False, legend_loc='on data')

In [ ]:
sc.pl.umap(adata_reynolds_fb_4820STDY7388991_loom_fb, color=['leiden', 'SFRP2', 'PI16', 'SLPI', 'WIF1', 'COL18A1', 'COMP', 
                                               'APOE', 'CCL2',  'ITM2A', 'SPSB1', 
                                               'CCDC146', 'CCL19', 'CD74', 
                                               'DKK3', 'TNN', 'SFRP1', 'POSTN'], cmap=magma, use_raw=False, legend_loc='on data')

## 4820STDY7388999 (S2)

### Reynolds preprocessed

In [ ]:
adata_reynolds_fb_4820STDY7388999 = adata_reynolds_fb_healthy[adata_reynolds_fb_healthy.obs['sample_id'].isin(['4820STDY7388999'])]  #selected_samples)]

In [ ]:
adata_reynolds_fb_4820STDY7388999

In [ ]:
sc.pp.filter_genes(adata_reynolds_fb_4820STDY7388999, min_counts=1)
adata_reynolds_fb_4820STDY7388999 = adata_reynolds_fb_4820STDY7388999[:, adata_reynolds_fb_4820STDY7388999.var_names != 'C3']

In [ ]:
sc.pp.pca(adata_reynolds_fb_4820STDY7388999, random_state=seed, n_comps=30)
tk.tl.triku(adata_reynolds_fb_4820STDY7388999, n_procs=1, random_state=seed, use_adata_knn=True)
sc.pp.neighbors(adata_reynolds_fb_4820STDY7388999, metric='cosine', random_state=seed)

In [ ]:
sc.tl.umap(adata_reynolds_fb_4820STDY7388999, min_dist=0.3, random_state=seed)

In [ ]:
sc.tl.leiden(adata_reynolds_fb_4820STDY7388999, resolution=1, random_state=seed)

In [ ]:
sc.pl.umap(adata_reynolds_fb_4820STDY7388999, color=['leiden', 'sample_id', 'full_clustering'], legend_loc='on data')

In [ ]:
sc.pl.umap(adata_reynolds_fb_4820STDY7388999, color=['APCDD1', 'COL18A1', 'COMP', 'SLPI', 'WIF1'], cmap=magma, use_raw=False)

In [ ]:
sc.pl.umap(adata_reynolds_fb_4820STDY7388999, color=['MT2A', 'CCL19', 'CCL2', 'CD46'], cmap=magma, use_raw=False)

In [ ]:
sc.pl.umap(adata_reynolds_fb_4820STDY7388999, color=['POSTN', 'COMP', 'COCH'], cmap=magma, use_raw=False)

### Direct loom

In [ ]:
adata_reynolds_fb_4820STDY7388999_loom = sc.read('reynolds_2020/reynolds_2020_8_4820STDY7388999_s2_dermis_fibroblasts.loom', 
                                                 backup_url='https://zenodo.org/record/4596375/files/reynolds_2020_8_4820STDY7388999_s2_dermis_fibroblasts.loom')

adata_reynolds_fb_4820STDY7388999_loom.var_names_make_unique()
adata_reynolds_fb_4820STDY7388999_loom = adata_reynolds_fb_4820STDY7388999_loom[:, adata_reynolds_fb_4820STDY7388999_loom.var_names != 'C3']

In [ ]:
adata_reynolds_fb_4820STDY7388999_loom.obs_names = [f"{i.split('_')[-1]}-1-{i.split('_')[1]}" for i in adata_reynolds_fb_4820STDY7388999_loom.obs_names]

In [ ]:
df_fb_type = pd.Series('Other', index=adata_reynolds_fb_4820STDY7388999_loom.obs_names)
df_fb_type.loc[adata_reynolds_fb_4820STDY7388999.obs_names & adata_reynolds_fb_4820STDY7388999_loom.obs_names] = adata_reynolds_fb_4820STDY7388999[adata_reynolds_fb_4820STDY7388999.obs_names & adata_reynolds_fb_4820STDY7388999_loom.obs_names].obs['full_clustering']
adata_reynolds_fb_4820STDY7388999_loom.obs['science_clustering'] = df_fb_type

In [ ]:
sc.pp.calculate_qc_metrics(adata_reynolds_fb_4820STDY7388999_loom, inplace=True)

In [ ]:
sc.pl.violin(adata_reynolds_fb_4820STDY7388999_loom, ['n_genes_by_counts'])
sc.pl.violin(adata_reynolds_fb_4820STDY7388999_loom, ['log1p_total_counts'])

In [ ]:
sc.pp.filter_genes(adata_reynolds_fb_4820STDY7388999_loom, min_counts=30)
adata_reynolds_fb_4820STDY7388999_loom.raw = adata_reynolds_fb_4820STDY7388999_loom

In [ ]:
sc.pp.normalize_per_cell(adata_reynolds_fb_4820STDY7388999_loom)
sc.pp.log1p(adata_reynolds_fb_4820STDY7388999_loom)

In [ ]:
sc.pp.pca(adata_reynolds_fb_4820STDY7388999_loom, random_state=seed, n_comps=30)
tk.tl.triku(adata_reynolds_fb_4820STDY7388999_loom, n_procs=1, random_state=seed, use_adata_knn=True)
sc.pp.neighbors(adata_reynolds_fb_4820STDY7388999_loom, metric='cosine', random_state=seed)

In [ ]:
sc.tl.umap(adata_reynolds_fb_4820STDY7388999_loom, min_dist=0.3, random_state=seed)
sc.tl.leiden(adata_reynolds_fb_4820STDY7388999_loom, resolution=0.5, random_state=seed)

In [ ]:
sc.pl.umap(adata_reynolds_fb_4820STDY7388999_loom, color=['leiden', 'science_clustering'], legend_loc='on data')

In [ ]:
sc.pl.umap(adata_reynolds_fb_4820STDY7388999_loom, color=['LUM', 'PDGFRA', 'VIM', 'COL1A1', 'SFRP2', 'APOE', 'POSTN'], legend_loc='on data', cmap=magma)

In [ ]:
sc.pl.umap(adata_reynolds_fb_4820STDY7388999_loom, color=['leiden', 'SFRP2', 'PI16', 'SLPI', 'WIF1', 'COL18A1', 'COMP', 
                                               'APOE', 'CCL2',  'ITM2A', 'SPSB1', 
                                               'CCDC146', 'CCL19', 'CD74', 
                                               'DKK3', 'TNN', 'SFRP1'], cmap=magma, use_raw=False, legend_loc='on data')

In [ ]:
sc.pl.umap(adata_reynolds_fb_4820STDY7388999_loom, color=['leiden', 
                                                          'DMKN', 'KRT1'], cmap=magma, use_raw=False, legend_loc='on data')

In [ ]:
dict_cat = {'fb': ['LUM', 'PDGFRA', 'COL1A1', 'SFRP2', 'CCL19'], 'peri': ['RGS5', 'MYL9', 'NDUFA4L2'], 'eritro': ['HBB', 'HBA2', 'HBA1'], 
            'immune': ['TPSB2', 'TRAC', 'TPSAB1', 'HLA-DRA', 'FCER1G', 'CD74'], 'endo': ['CLDN5', 'PECAM1'], 'kerato': ['DMKN', 'KRT1'],
            'mt': ['MTND2P28', 'MTND4P12', 'MTCO1P40', 'ADAM33', 'RN7SL2', 'MTRNR2L6']}

assign_cats(adata_reynolds_fb_4820STDY7388999_loom, dict_cat)

In [ ]:
sc.pl.umap(adata_reynolds_fb_4820STDY7388999_loom, color=['leiden', 'science_clustering', 'assigned_cats'], legend_loc='on data', cmap=magma)

In [ ]:
sc.tl.rank_genes_groups(adata_reynolds_fb_4820STDY7388999_loom, groupby='leiden')
sc.pl.rank_genes_groups(adata_reynolds_fb_4820STDY7388999_loom)

In [ ]:
adata_reynolds_fb_4820STDY7388999_loom = adata_reynolds_fb_4820STDY7388999_loom[adata_reynolds_fb_4820STDY7388999_loom.obs['assigned_cats'] == 'fb']

In [ ]:
sc.pp.filter_genes(adata_reynolds_fb_4820STDY7388999_loom, min_counts=1)

In [ ]:
sc.pp.pca(adata_reynolds_fb_4820STDY7388999_loom, random_state=seed, n_comps=30)
tk.tl.triku(adata_reynolds_fb_4820STDY7388999_loom, n_procs=1, random_state=seed, use_adata_knn=True)
sc.pp.neighbors(adata_reynolds_fb_4820STDY7388999_loom, metric='cosine', random_state=seed)

In [ ]:
sc.tl.umap(adata_reynolds_fb_4820STDY7388999_loom, min_dist=0.05, random_state=seed)
sc.tl.leiden(adata_reynolds_fb_4820STDY7388999_loom, resolution=1.2, random_state=seed)

In [ ]:
sc.pl.umap(adata_reynolds_fb_4820STDY7388999_loom, color=['leiden', 'science_clustering'], cmap=magma, use_raw=False, legend_loc='on data')

In [ ]:
sc.pl.umap(adata_reynolds_fb_4820STDY7388999_loom, color=['leiden', 'SFRP2', 'PI16', 'SLPI', 'WIF1', 'COL18A1', 'COMP', 
                                               'APOE', 'CCL2',  'ITM2A', 'SPSB1', 
                                               'CCDC146', 'CCL19', 'CD74', 
                                               'DKK3', 'TNN', 'SFRP1', 'EDNRB', 'IGFBP7'], cmap=magma, use_raw=False, legend_loc='on data')

In [ ]:
sc.tl.rank_genes_groups(adata_reynolds_fb_4820STDY7388999_loom, groupby='leiden')

In [ ]:
sc.pl.rank_genes_groups(adata_reynolds_fb_4820STDY7388999_loom)

## 4820STDY7389007 (S3)

### Reynolds preprocessed

In [ ]:
adata_reynolds_fb_4820STDY7389007 = adata_reynolds_fb_healthy[adata_reynolds_fb_healthy.obs['sample_id'].isin(['4820STDY7389007'])]  #selected_samples)]

In [ ]:
adata_reynolds_fb_4820STDY7389007

In [ ]:
sc.pp.filter_genes(adata_reynolds_fb_4820STDY7389007, min_counts=1)
adata_reynolds_fb_4820STDY7389007 = adata_reynolds_fb_4820STDY7389007[:, adata_reynolds_fb_4820STDY7389007.var_names != 'C3']

In [ ]:
sc.pp.pca(adata_reynolds_fb_4820STDY7389007, random_state=seed, n_comps=30)
tk.tl.triku(adata_reynolds_fb_4820STDY7389007, n_procs=1, random_state=seed, use_adata_knn=True)
sc.pp.neighbors(adata_reynolds_fb_4820STDY7389007, metric='cosine', random_state=seed)

In [ ]:
sc.tl.umap(adata_reynolds_fb_4820STDY7389007, min_dist=0.3, random_state=seed)

In [ ]:
sc.tl.leiden(adata_reynolds_fb_4820STDY7389007, resolution=1, random_state=seed)

In [ ]:
sc.pl.umap(adata_reynolds_fb_4820STDY7389007, color=['leiden', 'sample_id', 'full_clustering'], legend_loc='on data')

In [ ]:
sc.pl.umap(adata_reynolds_fb_4820STDY7389007, color=['APCDD1', 'COL18A1', 'COMP', 'SLPI', 'WIF1'], cmap=magma, use_raw=False)

In [ ]:
sc.pl.umap(adata_reynolds_fb_4820STDY7389007, color=['MT2A', 'CCL19', 'CCL2', 'CD46'], cmap=magma, use_raw=False)

In [ ]:
sc.pl.umap(adata_reynolds_fb_4820STDY7389007, color=['POSTN', 'COMP', 'COCH'], cmap=magma, use_raw=False)

### Direct loom

In [ ]:
adata_reynolds_fb_4820STDY7389007_loom = sc.read('reynolds_2020/reynolds_2020_16_4820STDY7389007_s3_dermis_fibroblasts.loom', 
                                                 backup_url='https://zenodo.org/record/4596375/files/reynolds_2020_16_4820STDY7389007_s3_dermis_fibroblasts.loom')

adata_reynolds_fb_4820STDY7389007_loom.var_names_make_unique()

adata_reynolds_fb_4820STDY7389007_loom = adata_reynolds_fb_4820STDY7389007_loom[:, adata_reynolds_fb_4820STDY7389007_loom.var_names != 'C3']

In [ ]:
adata_reynolds_fb_4820STDY7389007_loom.obs_names = [f"{i.split('_')[-1]}-1-{i.split('_')[1]}" for i in adata_reynolds_fb_4820STDY7389007_loom.obs_names]

In [ ]:
df_fb_type = pd.Series('-', index=adata_reynolds_fb_4820STDY7389007_loom.obs_names)
df_fb_type.loc[adata_reynolds_fb_4820STDY7389007.obs_names & adata_reynolds_fb_4820STDY7389007_loom.obs_names] = adata_reynolds_fb_4820STDY7389007[adata_reynolds_fb_4820STDY7389007.obs_names & adata_reynolds_fb_4820STDY7389007_loom.obs_names].obs['full_clustering']
adata_reynolds_fb_4820STDY7389007_loom.obs['science_clustering'] = df_fb_type

In [ ]:
sc.pp.calculate_qc_metrics(adata_reynolds_fb_4820STDY7389007_loom, inplace=True)

In [ ]:
sc.pl.violin(adata_reynolds_fb_4820STDY7389007_loom, ['n_genes_by_counts'])
sc.pl.violin(adata_reynolds_fb_4820STDY7389007_loom, ['log1p_total_counts'])

In [ ]:
sc.pp.filter_genes(adata_reynolds_fb_4820STDY7389007_loom, min_counts=30)
adata_reynolds_fb_4820STDY7389007_loom.raw = adata_reynolds_fb_4820STDY7389007_loom

In [ ]:
sc.pp.normalize_per_cell(adata_reynolds_fb_4820STDY7389007_loom)
sc.pp.log1p(adata_reynolds_fb_4820STDY7389007_loom)

In [ ]:
sc.pp.pca(adata_reynolds_fb_4820STDY7389007_loom, random_state=seed, n_comps=30)
tk.tl.triku(adata_reynolds_fb_4820STDY7389007_loom, n_procs=1, random_state=seed, use_adata_knn=True)
sc.pp.neighbors(adata_reynolds_fb_4820STDY7389007_loom, metric='cosine', random_state=seed)

In [ ]:
sc.tl.umap(adata_reynolds_fb_4820STDY7389007_loom, min_dist=0.3, random_state=seed)
sc.tl.leiden(adata_reynolds_fb_4820STDY7389007_loom, resolution=0.8, random_state=seed)

In [ ]:
sc.pl.umap(adata_reynolds_fb_4820STDY7389007_loom, color=['leiden', 'science_clustering'], legend_loc='on data')

In [ ]:
sc.pl.umap(adata_reynolds_fb_4820STDY7389007_loom, color=['LUM', 'PDGFRA', 'VIM', 'COL1A1', 'SFRP2', 'APOE', 'POSTN', 'RGS5', 'MYL9', 'NDUFA4L2', 'HBB'], legend_loc='on data', cmap=magma)

In [ ]:
sc.pl.umap(adata_reynolds_fb_4820STDY7389007_loom, color=['leiden', 'SFRP2', 'PI16', 'SLPI', 'WIF1', 'COL18A1', 'COMP', 
                                               'APOE', 'CCL2',  'ITM2A', 'SPSB1', 
                                               'CCDC146', 'CCL19', 'CD74'], cmap=magma, use_raw=False, legend_loc='on data')

In [ ]:
dict_cat = {'fb': ['LUM', 'PDGFRA', 'COL1A1', 'SFRP2', 'CCL19'], 'peri': ['RGS5', 'MYL9', 'NDUFA4L2'], 'eritro': ['HBB', 'HBA2', 'HBA1'], 
            'immune': ['TPSB2', 'TRAC', 'TPSAB1', 'HLA-DRA', 'FCER1G'], 'kerato': ['DMKN', 'KRT1'],
            'mt': ['MTND2P28', 'MTND4P12', 'MTCO1P40', 'ADAM33', 'RN7SL2', 'MTRNR2L6']}

assign_cats(adata_reynolds_fb_4820STDY7389007_loom, dict_cat)

In [ ]:
sc.pl.umap(adata_reynolds_fb_4820STDY7389007_loom, color=['leiden', 'science_clustering', 'assigned_cats'], legend_loc='on data', cmap=magma)

In [ ]:
sc.tl.rank_genes_groups(adata_reynolds_fb_4820STDY7389007_loom, groupby='leiden')
sc.pl.rank_genes_groups(adata_reynolds_fb_4820STDY7389007_loom)

In [ ]:
adata_reynolds_fb_4820STDY7389007_loom = adata_reynolds_fb_4820STDY7389007_loom[adata_reynolds_fb_4820STDY7389007_loom.obs['assigned_cats'] == 'fb']

In [ ]:
sc.pp.filter_genes(adata_reynolds_fb_4820STDY7389007_loom, min_counts=1)

In [ ]:
sc.pp.pca(adata_reynolds_fb_4820STDY7389007_loom, random_state=seed, n_comps=30)
tk.tl.triku(adata_reynolds_fb_4820STDY7389007_loom, n_procs=1, random_state=seed, use_adata_knn=True)
sc.pp.neighbors(adata_reynolds_fb_4820STDY7389007_loom, metric='cosine', random_state=seed)

In [ ]:
sc.tl.umap(adata_reynolds_fb_4820STDY7389007_loom, min_dist=0.05, random_state=seed)
sc.tl.leiden(adata_reynolds_fb_4820STDY7389007_loom, resolution=1.2, random_state=seed)

In [ ]:
adata_reynolds_fb_4820STDY7389007_loom

In [ ]:
sc.pl.umap(adata_reynolds_fb_4820STDY7389007_loom, color=['leiden', 'science_clustering'], cmap=magma, use_raw=False, legend_loc='on data')

In [ ]:
sc.pl.umap(adata_reynolds_fb_4820STDY7389007_loom, color=['leiden', 'SFRP2', 'PI16', 'SLPI', 'WIF1', 'COL18A1', 'COMP', 
                                               'APOE', 'CCL2',  'ITM2A', 'SPSB1', 
                                               'CCDC146', 'CCL19', 'CD74'], cmap=magma, use_raw=False, legend_loc='on data')

## SKN8104899 (S4)

### Reynolds preprocessed

In [ ]:
adata_reynolds_fb_SKN8104899 = adata_reynolds_fb_healthy[adata_reynolds_fb_healthy.obs['sample_id'].isin(['SKN8104899'])]  #selected_samples)]

In [ ]:
adata_reynolds_fb_SKN8104899

In [ ]:
sc.pp.filter_genes(adata_reynolds_fb_SKN8104899, min_counts=1)
adata_reynolds_fb_SKN8104899 = adata_reynolds_fb_SKN8104899[:, adata_reynolds_fb_SKN8104899.var_names != 'C3']

In [ ]:
sc.pp.pca(adata_reynolds_fb_SKN8104899, random_state=seed, n_comps=30)
tk.tl.triku(adata_reynolds_fb_SKN8104899, n_procs=1, random_state=seed, use_adata_knn=True)
sc.pp.neighbors(adata_reynolds_fb_SKN8104899, metric='cosine', random_state=seed)

In [ ]:
sc.tl.umap(adata_reynolds_fb_SKN8104899, min_dist=0.3, random_state=seed)

In [ ]:
sc.tl.leiden(adata_reynolds_fb_SKN8104899, resolution=1, random_state=seed)

In [ ]:
sc.pl.umap(adata_reynolds_fb_SKN8104899, color=['leiden', 'sample_id', 'full_clustering'], legend_loc='on data')

In [ ]:
sc.pl.umap(adata_reynolds_fb_SKN8104899, color=['APCDD1', 'COL18A1', 'COMP', 'SLPI', 'WIF1'], cmap=magma, use_raw=False)

In [ ]:
sc.pl.umap(adata_reynolds_fb_SKN8104899, color=['MT2A', 'CCL19', 'CCL2', 'CD46'], cmap=magma, use_raw=False)

### Direct loom

In [ ]:
adata_reynolds_fb_SKN8104899_loom = sc.read('reynolds_2020/reynolds_2020_84_SKN8104899_S4_dermis_fibroblasts.loom', 
                                                 backup_url='https://zenodo.org/record/4596375/files/reynolds_2020_84_SKN8104899_S4_dermis_fibroblasts.loom')

adata_reynolds_fb_SKN8104899_loom.var_names_make_unique()

adata_reynolds_fb_SKN8104899_loom = adata_reynolds_fb_SKN8104899_loom[:, adata_reynolds_fb_SKN8104899_loom.var_names != 'C3']

In [ ]:
adata_reynolds_fb_SKN8104899_loom.obs_names = [f"{i.split('_')[-1]}-1-{i.split('_')[1]}" for i in adata_reynolds_fb_SKN8104899_loom.obs_names]

In [ ]:
df_fb_type = pd.Series('-', index=adata_reynolds_fb_SKN8104899_loom.obs_names)
df_fb_type.loc[adata_reynolds_fb_SKN8104899.obs_names & adata_reynolds_fb_SKN8104899_loom.obs_names] = adata_reynolds_fb_SKN8104899[adata_reynolds_fb_SKN8104899.obs_names & adata_reynolds_fb_SKN8104899_loom.obs_names].obs['full_clustering']
adata_reynolds_fb_SKN8104899_loom.obs['science_clustering'] = df_fb_type

In [ ]:
sc.pp.calculate_qc_metrics(adata_reynolds_fb_SKN8104899_loom, inplace=True)

In [ ]:
sc.pl.violin(adata_reynolds_fb_SKN8104899_loom, ['n_genes_by_counts'])
sc.pl.violin(adata_reynolds_fb_SKN8104899_loom, ['log1p_total_counts'])

In [ ]:
sc.pp.filter_genes(adata_reynolds_fb_SKN8104899_loom, min_counts=30)
adata_reynolds_fb_SKN8104899_loom.raw = adata_reynolds_fb_SKN8104899_loom

In [ ]:
sc.pp.normalize_per_cell(adata_reynolds_fb_SKN8104899_loom)
sc.pp.log1p(adata_reynolds_fb_SKN8104899_loom)

In [ ]:
sc.pp.pca(adata_reynolds_fb_SKN8104899_loom, random_state=seed, n_comps=30)
tk.tl.triku(adata_reynolds_fb_SKN8104899_loom, n_procs=1, random_state=seed, use_adata_knn=True)
sc.pp.neighbors(adata_reynolds_fb_SKN8104899_loom, metric='cosine', random_state=seed)

In [ ]:
sc.tl.umap(adata_reynolds_fb_SKN8104899_loom, min_dist=0.3, random_state=seed)
sc.tl.leiden(adata_reynolds_fb_SKN8104899_loom, resolution=0.8, random_state=seed)

In [ ]:
sc.pl.umap(adata_reynolds_fb_SKN8104899_loom, color=['leiden', 'science_clustering'], legend_loc='on data')

In [ ]:
sc.pl.umap(adata_reynolds_fb_SKN8104899_loom, color=['LUM', 'PDGFRA', 'VIM', 'COL1A1', 'SFRP2', 'APOE', 'POSTN', 'RGS5', 'MYL9', 'NDUFA4L2', 'HBB'], legend_loc='on data', cmap=magma)

In [ ]:
sc.pl.umap(adata_reynolds_fb_SKN8104899_loom, color=['leiden', 'SFRP2', 'PI16', 'SLPI', 'WIF1', 'COL18A1', 'COMP', 
                                               'APOE', 'CCL2',  'ITM2A', 'SPSB1', 
                                               'CCDC146', 'CCL19', 'CD74', 'POSTN'], cmap=magma, use_raw=False, legend_loc='on data')

In [ ]:
dict_cat = {'fb': ['LUM', 'PDGFRA', 'COL1A1', 'SFRP2', 'CCL19'], 'peri': ['RGS5', 'MYL9', 'NDUFA4L2'], 'eritro': ['HBB', 'HBA2', 'HBA1'], 
            'immune': ['TRAC', 'HLA-DRA', 'FCER1G'], 'kerato': ['DMKN', 'KRT1'],
            'mt': ['MTND4P12', 'ADAM33', 'RN7SL2', ]}

assign_cats(adata_reynolds_fb_SKN8104899_loom, dict_cat)

In [ ]:
sc.pl.umap(adata_reynolds_fb_SKN8104899_loom, color=['leiden', 'science_clustering', 'assigned_cats'], legend_loc='on data', cmap=magma)

In [ ]:
sc.tl.rank_genes_groups(adata_reynolds_fb_SKN8104899_loom, groupby='leiden')
sc.pl.rank_genes_groups(adata_reynolds_fb_SKN8104899_loom)

In [ ]:
adata_reynolds_fb_SKN8104899_loom = adata_reynolds_fb_SKN8104899_loom[adata_reynolds_fb_SKN8104899_loom.obs['assigned_cats'] == 'fb']

In [ ]:
sc.pp.filter_genes(adata_reynolds_fb_SKN8104899_loom, min_counts=1)

In [ ]:
sc.pp.pca(adata_reynolds_fb_SKN8104899_loom, random_state=seed, n_comps=30)
tk.tl.triku(adata_reynolds_fb_SKN8104899_loom, n_procs=1, random_state=seed, use_adata_knn=True)
sc.pp.neighbors(adata_reynolds_fb_SKN8104899_loom, metric='cosine', random_state=seed)

In [ ]:
sc.tl.umap(adata_reynolds_fb_SKN8104899_loom, min_dist=0.05, random_state=seed)
sc.tl.leiden(adata_reynolds_fb_SKN8104899_loom, resolution=1.2, random_state=seed)

In [ ]:
sc.pl.umap(adata_reynolds_fb_SKN8104899_loom, color=['leiden', 'science_clustering'], cmap=magma, use_raw=False, legend_loc='on data')

In [ ]:
sc.pl.umap(adata_reynolds_fb_SKN8104899_loom, color=['leiden', 'SFRP2', 'PI16', 'SLPI', 'WIF1', 'COL18A1', 'COMP', 
                                               'APOE', 'CCL2',  'ITM2A', 'SPSB1', 
                                               'CCDC146', 'CCL19', 'CD74'], cmap=magma, use_raw=False, legend_loc='on data')

In [ ]:
sc.tl.rank_genes_groups(adata_reynolds_fb_SKN8104899_loom, groupby='leiden')
sc.pl.rank_genes_groups(adata_reynolds_fb_SKN8104899_loom)

## SKN8105197 (S5)

### Reynolds preprocessed

In [ ]:
adata_reynolds_fb_SKN8105197 = adata_reynolds_fb_healthy[adata_reynolds_fb_healthy.obs['sample_id'].isin(['SKN8105197'])]  #selected_samples)]

In [ ]:
adata_reynolds_fb_SKN8105197

In [ ]:
sc.pp.filter_genes(adata_reynolds_fb_SKN8105197, min_counts=1)
adata_reynolds_fb_SKN8105197 = adata_reynolds_fb_SKN8105197[:, adata_reynolds_fb_SKN8105197.var_names != 'C3']

In [ ]:
sc.pp.pca(adata_reynolds_fb_SKN8105197, random_state=seed, n_comps=30)
tk.tl.triku(adata_reynolds_fb_SKN8105197, n_procs=1, random_state=seed, use_adata_knn=True)
sc.pp.neighbors(adata_reynolds_fb_SKN8105197, metric='cosine', random_state=seed)

In [ ]:
sc.tl.umap(adata_reynolds_fb_SKN8105197, min_dist=0.3, random_state=seed)

In [ ]:
sc.tl.leiden(adata_reynolds_fb_SKN8105197, resolution=1, random_state=seed)

In [ ]:
sc.pl.umap(adata_reynolds_fb_SKN8105197, color=['leiden', 'sample_id', 'full_clustering'], legend_loc='on data')

In [ ]:
sc.pl.umap(adata_reynolds_fb_SKN8105197, color=['APCDD1', 'COL18A1', 'COMP', 'SLPI', 'WIF1'], cmap=magma, use_raw=False)

In [ ]:
sc.pl.umap(adata_reynolds_fb_SKN8105197, color=['MT2A', 'CCL19', 'CCL2', 'CD46'], cmap=magma, use_raw=False)

### Direct loom

In [ ]:
adata_reynolds_fb_SKN8105197_loom = sc.read('reynolds_2020/reynolds_2020_92_SKN8105197_S5_dermis_fibroblasts.loom', 
                                                 backup_url='https://zenodo.org/record/4596375/files/reynolds_2020_92_SKN8105197_S5_dermis_fibroblasts.loom')
adata_reynolds_fb_SKN8105197_loom.var_names_make_unique()

adata_reynolds_fb_SKN8105197_loom = adata_reynolds_fb_SKN8105197_loom[:, adata_reynolds_fb_SKN8105197_loom.var_names != 'C3']

In [ ]:
adata_reynolds_fb_SKN8105197_loom.obs_names = [f"{i.split('_')[-1]}-1-{i.split('_')[1]}" for i in adata_reynolds_fb_SKN8105197_loom.obs_names]

In [ ]:
df_fb_type = pd.Series('-', index=adata_reynolds_fb_SKN8105197_loom.obs_names)
df_fb_type.loc[adata_reynolds_fb_SKN8105197.obs_names & adata_reynolds_fb_SKN8105197_loom.obs_names] = adata_reynolds_fb_SKN8105197[adata_reynolds_fb_SKN8105197.obs_names & adata_reynolds_fb_SKN8105197_loom.obs_names].obs['full_clustering']
adata_reynolds_fb_SKN8105197_loom.obs['science_clustering'] = df_fb_type

In [ ]:
sc.pp.calculate_qc_metrics(adata_reynolds_fb_SKN8105197_loom, inplace=True)

In [ ]:
sc.pl.violin(adata_reynolds_fb_SKN8105197_loom, ['n_genes_by_counts'])
sc.pl.violin(adata_reynolds_fb_SKN8105197_loom, ['log1p_total_counts'])

In [ ]:
sc.pp.filter_genes(adata_reynolds_fb_SKN8105197_loom, min_counts=30)
adata_reynolds_fb_SKN8105197_loom.raw = adata_reynolds_fb_SKN8105197_loom

In [ ]:
sc.pp.normalize_per_cell(adata_reynolds_fb_SKN8105197_loom)
sc.pp.log1p(adata_reynolds_fb_SKN8105197_loom)

In [ ]:
sc.pp.pca(adata_reynolds_fb_SKN8105197_loom, random_state=seed, n_comps=30)
tk.tl.triku(adata_reynolds_fb_SKN8105197_loom, n_procs=1, random_state=seed, use_adata_knn=True)
sc.pp.neighbors(adata_reynolds_fb_SKN8105197_loom, metric='cosine', random_state=seed)

In [ ]:
sc.tl.umap(adata_reynolds_fb_SKN8105197_loom, min_dist=0.3, random_state=seed)
sc.tl.leiden(adata_reynolds_fb_SKN8105197_loom, resolution=1, random_state=seed)

In [ ]:
sc.pl.umap(adata_reynolds_fb_SKN8105197_loom, color=['leiden', 'science_clustering'], legend_loc='on data')

In [ ]:
sc.pl.umap(adata_reynolds_fb_SKN8105197_loom, color=['LUM', 'PDGFRA', 'VIM', 'COL1A1', 'SFRP2', 'APOE', 'RGS5', 'MYL9', 'NDUFA4L2', 'HBB'], legend_loc='on data', cmap=magma)

In [ ]:
sc.pl.umap(adata_reynolds_fb_SKN8105197_loom, color=['leiden', 'SFRP2', 'SLPI', 'WIF1', 'COL18A1', 'COMP', 
                                               'APOE', 'CCL2',  'ITM2A', 'SPSB1', 
                                               'CCDC146', 'CCL19', 'CD74'], cmap=magma, use_raw=False, legend_loc='on data')

In [ ]:
dict_cat = {'fb': ['LUM', 'PDGFRA', 'COL1A1', 'SFRP2', 'CCL19'], 'peri': ['RGS5', 'MYL9', 'NDUFA4L2'], 'eritro': ['HBB', 'HBA2', 'HBA1'], 
            'immune': ['HLA-DRA', 'FCER1G'], 'melano':['PMEL', 'MLANA']}

assign_cats(adata_reynolds_fb_SKN8105197_loom, dict_cat)

In [ ]:
sc.pl.umap(adata_reynolds_fb_SKN8105197_loom, color=['leiden', 'science_clustering', 'assigned_cats'], legend_loc='on data', cmap=magma)

In [ ]:
sc.tl.rank_genes_groups(adata_reynolds_fb_SKN8105197_loom, groupby='leiden')
sc.pl.rank_genes_groups(adata_reynolds_fb_SKN8105197_loom)

In [ ]:
adata_reynolds_fb_SKN8105197_loom = adata_reynolds_fb_SKN8105197_loom[adata_reynolds_fb_SKN8105197_loom.obs['assigned_cats'] == 'fb']

In [ ]:
sc.pp.filter_genes(adata_reynolds_fb_SKN8105197_loom, min_counts=1)

In [ ]:
sc.pp.pca(adata_reynolds_fb_SKN8105197_loom, random_state=seed, n_comps=30)
tk.tl.triku(adata_reynolds_fb_SKN8105197_loom, n_procs=1, random_state=seed, use_adata_knn=True)
sc.pp.neighbors(adata_reynolds_fb_SKN8105197_loom, metric='cosine', random_state=seed)

In [ ]:
sc.tl.umap(adata_reynolds_fb_SKN8105197_loom, min_dist=0.05, random_state=seed)
sc.tl.leiden(adata_reynolds_fb_SKN8105197_loom, resolution=1.2, random_state=seed)

In [ ]:
sc.pl.umap(adata_reynolds_fb_SKN8105197_loom, color=['leiden', 'science_clustering'], cmap=magma, use_raw=False, legend_loc='on data')

In [ ]:
sc.pl.umap(adata_reynolds_fb_SKN8105197_loom, color=['leiden', 'SFRP2', 'SLPI', 'WIF1', 'COL18A1', 'COMP', 
                                               'APOE', 'CCL2',  'ITM2A', 'SPSB1', 
                                               'CCDC146', 'CCL19', 'CD74'], cmap=magma, use_raw=False, legend_loc='on data')

## SKN8105197 exclusion
We do not see SFRP2 expression, which has been a through fb marker, together with APOE. We will not include this sample in the analysis.

In [ ]:
genes = ['science_clustering', 'SFRP2', 'APOE', 'CCL19']

sc.pl.umap(adata_reynolds_fb_4820STDY7388991_loom_fb, color=genes, cmap=magma, use_raw=False, legend_loc='on data')
sc.pl.umap(adata_reynolds_fb_4820STDY7388999_loom, color=genes, cmap=magma, use_raw=False, legend_loc='on data')
sc.pl.umap(adata_reynolds_fb_4820STDY7389007_loom, color=genes, cmap=magma, use_raw=False, legend_loc='on data')

sc.pl.umap(adata_reynolds_fb_SKN8104899_loom, color=genes, cmap=magma, use_raw=False, legend_loc='on data')
sc.pl.umap(adata_reynolds_fb_SKN8105197_loom, color=genes, cmap=magma, use_raw=False, legend_loc='on data')

# Preparing adatas for FB1+3 and FB2

We've realized that FB1+3 and FB2 showed the same main populations based on JID (A1, A2 and B2 mainly). We want to obtain distinct populations based on the marker expression, and also be able to separate these two populations. The problem is that the FB1+3 and FB2 populations, although not fully overlapping, they do show some overlap between them, and we would like to separate them only based on marker genes and clustering. These markers should be shared between all datasets. To do that, we are going to do a preeliminar DEG analysis and find markers that separate these populations in an unbiased manner.

## Finding markers to separate 1+3 from 2

**This part may not be reproducible in other notebooks. This part is to get markers that will be reproducible later on**

In [ ]:
# S1
sc.tl.leiden(adata_reynolds_fb_4820STDY7388991_loom_fb, resolution=3, random_state=seed)
sc.pl.umap(adata_reynolds_fb_4820STDY7388991_loom_fb, color=['leiden', 'science_clustering'], cmap=magma, use_raw=False, legend_loc='on data')
adata_reynolds_fb_4820STDY7388991_loom_fb.obs['fb_type'] = ['1+3' if i in ['3', '16', '12', '19', '6', '17', '8', '2', '9', '11', '12', '14', '0', '21', '22'] else '2' for i in adata_reynolds_fb_4820STDY7388991_loom_fb.obs['leiden']]
sc.pl.umap(adata_reynolds_fb_4820STDY7388991_loom_fb, color=['fb_type', 'science_clustering'], cmap=magma, use_raw=False, legend_loc='on data')

In [ ]:
sc.tl.rank_genes_groups(adata_reynolds_fb_4820STDY7388991_loom_fb, groupby='fb_type', groups=['1+3'], reference='2', rankby_abs=True)
df_rank_s1 = pd.DataFrame(adata_reynolds_fb_4820STDY7388991_loom_fb.uns['rank_genes_groups']['names']['1+3'],
adata_reynolds_fb_4820STDY7388991_loom_fb.uns['rank_genes_groups']['scores']['1+3']).sort_index(ascending=False)

In [ ]:
# S2
sc.tl.leiden(adata_reynolds_fb_4820STDY7388999_loom, resolution=3, random_state=seed)
sc.pl.umap(adata_reynolds_fb_4820STDY7388999_loom, color=['leiden', 'science_clustering'], cmap=magma, use_raw=False, legend_loc='on data')
adata_reynolds_fb_4820STDY7388999_loom.obs['fb_type'] = ['2' if i in ['15', '16', '19'] else '1+3' for i in adata_reynolds_fb_4820STDY7388999_loom.obs['leiden']]
sc.pl.umap(adata_reynolds_fb_4820STDY7388999_loom, color=['fb_type', 'science_clustering'], cmap=magma, use_raw=False, legend_loc='on data')

In [ ]:
sc.tl.rank_genes_groups(adata_reynolds_fb_4820STDY7388999_loom, groupby='fb_type', groups=['1+3'], reference='2', rankby_abs=True)
df_rank_s2 = pd.DataFrame(adata_reynolds_fb_4820STDY7388999_loom.uns['rank_genes_groups']['names']['1+3'],
adata_reynolds_fb_4820STDY7388999_loom.uns['rank_genes_groups']['scores']['1+3']).sort_index(ascending=False)

In [ ]:
# S3
sc.tl.leiden(adata_reynolds_fb_4820STDY7389007_loom, resolution=2.8, random_state=seed)
sc.pl.umap(adata_reynolds_fb_4820STDY7389007_loom, color=['leiden', 'science_clustering'], cmap=magma, use_raw=False, legend_loc='on data')
adata_reynolds_fb_4820STDY7389007_loom.obs['fb_type'] = ['2' if i in ['0', '17', '16', '13', '4', '1', '19'] else '1+3' for i in adata_reynolds_fb_4820STDY7389007_loom.obs['leiden']]
sc.pl.umap(adata_reynolds_fb_4820STDY7389007_loom, color=['fb_type', 'science_clustering'], cmap=magma, use_raw=False, legend_loc='on data')

In [ ]:
sc.tl.rank_genes_groups(adata_reynolds_fb_4820STDY7389007_loom, groupby='fb_type', groups=['1+3'], reference='2', rankby_abs=True)
df_rank_s3 = pd.DataFrame(adata_reynolds_fb_4820STDY7389007_loom.uns['rank_genes_groups']['names']['1+3'],
adata_reynolds_fb_4820STDY7389007_loom.uns['rank_genes_groups']['scores']['1+3']).sort_index(ascending=False)

In [ ]:
# S4
sc.tl.leiden(adata_reynolds_fb_SKN8104899_loom, resolution=3, random_state=seed)
sc.pl.umap(adata_reynolds_fb_SKN8104899_loom, color=['leiden', 'science_clustering'], cmap=magma, use_raw=False, legend_loc='on data')
adata_reynolds_fb_SKN8104899_loom.obs['fb_type'] = ['2' if i in ['8', '6', '14', '27'] else '1+3' for i in adata_reynolds_fb_SKN8104899_loom.obs['leiden']]
sc.pl.umap(adata_reynolds_fb_SKN8104899_loom, color=['fb_type', 'science_clustering'], cmap=magma, use_raw=False, legend_loc='on data')

In [ ]:
sc.tl.rank_genes_groups(adata_reynolds_fb_SKN8104899_loom, groupby='fb_type', groups=['1+3'], reference='2', rankby_abs=True)
df_rank_s4 = pd.DataFrame(adata_reynolds_fb_SKN8104899_loom.uns['rank_genes_groups']['names']['1+3'],
adata_reynolds_fb_SKN8104899_loom.uns['rank_genes_groups']['scores']['1+3']).sort_index(ascending=False)

In [ ]:
N = 30

genes_13, counts_13 = np.unique(list(df_rank_s1.iloc[:N, 0].values) + list(df_rank_s2.iloc[:N, 0].values) + list(df_rank_s3.iloc[:N, 0].values) + list(df_rank_s4.iloc[:N, 0].values), return_counts=True)
genes_13_shared = genes_13[counts_13 >= 3]

genes_2, counts_2 = np.unique(list(df_rank_s1.iloc[-N:, 0].values) + list(df_rank_s2.iloc[-N:, 0].values) + list(df_rank_s3.iloc[-N:, 0].values) + list(df_rank_s4.iloc[-N:, 0].values), return_counts=True)
genes_2_shared = genes_2[counts_2 >= 3]

In [ ]:
genes_13_shared

In [ ]:
genes_2_shared

## Separe the 1+3 from 2

In [ ]:
genes_13_shared = ['BNIP3L', 'BNIP3P1', 'CEMIP', 'ENO1', 'GSTO1', 'MEDAG', 'NAMPT', 'NAMPTP1', 'NRN1', ]

In [ ]:
genes_2_shared = ['ANXA2', 'CALM1', 'FOSB', 'LMNA', 'MTRNR2L12', 'S100A10', 'S100A4', 'S100A6', 'TMSB4X', 'TPPP3']

In [ ]:
dict_cat = {'1+3': genes_13_shared, '2': genes_2_shared}

for adata in [adata_reynolds_fb_4820STDY7388991_loom_fb, adata_reynolds_fb_4820STDY7388999_loom, adata_reynolds_fb_4820STDY7389007_loom, adata_reynolds_fb_SKN8104899_loom]:
    assign_cats(adata, dict_cat)

In [ ]:
sc.pl.umap(adata_reynolds_fb_4820STDY7388991_loom_fb, color=['leiden', 'science_clustering', 'fb_type', 'assigned_cats',  
                                                             'assigned_cats_1+3', 'assigned_cats_2', 'assigned_cats_CV'], legend_loc='on data', cmap=magma)

In [ ]:
sc.pl.umap(adata_reynolds_fb_4820STDY7388991_loom_fb, color=['leiden'] + genes_13_shared, legend_loc='on data', cmap=magma, use_raw=False)

In [ ]:
sc.pl.umap(adata_reynolds_fb_4820STDY7388991_loom_fb, color=['leiden'] + genes_2_shared, legend_loc='on data', cmap=magma, use_raw=False)

In [ ]:
sc.pl.umap(adata_reynolds_fb_4820STDY7388999_loom, color=['leiden', 'science_clustering', 'fb_type', 'assigned_cats', 
                                                          'assigned_cats_1+3', 'assigned_cats_2', 'assigned_cats_CV'], legend_loc='on data', cmap=magma)

In [ ]:
sc.pl.umap(adata_reynolds_fb_4820STDY7388999_loom, color=['leiden'] + genes_13_shared, legend_loc='on data', cmap=magma, use_raw=False)

In [ ]:
sc.pl.umap(adata_reynolds_fb_4820STDY7388999_loom, color=['leiden'] + genes_2_shared, legend_loc='on data', cmap=magma, use_raw=False)

In [ ]:
sc.pl.umap(adata_reynolds_fb_4820STDY7389007_loom, color=['leiden', 'science_clustering', 'fb_type', 'assigned_cats',  
                                                          'assigned_cats_1+3', 'assigned_cats_2', 'assigned_cats_CV'], legend_loc='on data', cmap=magma)

In [ ]:
sc.pl.umap(adata_reynolds_fb_4820STDY7389007_loom, color=['leiden'] + genes_13_shared, legend_loc='on data', cmap=magma, use_raw=False)

In [ ]:
sc.pl.umap(adata_reynolds_fb_4820STDY7389007_loom, color=['leiden'] + genes_2_shared, legend_loc='on data', cmap=magma, use_raw=False)

In [ ]:
sc.pl.umap(adata_reynolds_fb_SKN8104899_loom, color=['leiden', 'science_clustering', 'fb_type', 'assigned_cats',  
                                                     'assigned_cats_1+3', 'assigned_cats_2', 'assigned_cats_CV'], legend_loc='on data', cmap=magma)

In [ ]:
sc.pl.umap(adata_reynolds_fb_SKN8104899_loom, color=['leiden'] + genes_13_shared, legend_loc='on data', cmap=magma, use_raw=False)

In [ ]:
sc.pl.umap(adata_reynolds_fb_SKN8104899_loom, color=['leiden'] + genes_2_shared, legend_loc='on data', cmap=magma, use_raw=False)

## Create the adatas

In [ ]:
adata_s1_13 = adata_reynolds_fb_4820STDY7388991_loom_fb[adata_reynolds_fb_4820STDY7388991_loom_fb.obs['assigned_cats'] == '1+3'].copy()
adata_s1_2 = adata_reynolds_fb_4820STDY7388991_loom_fb[adata_reynolds_fb_4820STDY7388991_loom_fb.obs['assigned_cats'] == '2'].copy()

adata_s2_13 = adata_reynolds_fb_4820STDY7388999_loom[adata_reynolds_fb_4820STDY7388999_loom.obs['assigned_cats'] == '1+3'].copy()
adata_s2_2 = adata_reynolds_fb_4820STDY7388999_loom[adata_reynolds_fb_4820STDY7388999_loom.obs['assigned_cats'] == '2'].copy()

adata_s3_13 = adata_reynolds_fb_4820STDY7389007_loom[adata_reynolds_fb_4820STDY7389007_loom.obs['assigned_cats'] == '1+3'].copy()
adata_s3_2 = adata_reynolds_fb_4820STDY7389007_loom[adata_reynolds_fb_4820STDY7389007_loom.obs['assigned_cats'] == '2'].copy()

adata_s4_13 = adata_reynolds_fb_SKN8104899_loom[adata_reynolds_fb_SKN8104899_loom.obs['assigned_cats'] == '1+3'].copy()
adata_s4_2 = adata_reynolds_fb_SKN8104899_loom[adata_reynolds_fb_SKN8104899_loom.obs['assigned_cats'] == '2'].copy()

In [ ]:
for adata_s_name in ['adata_s1_13', 'adata_s1_2', 'adata_s2_13', 'adata_s2_2', 'adata_s3_13', 'adata_s3_2', 'adata_s4_13', 'adata_s4_2']:  # We do this because with the adata name by itselft it won't work!!!
    sc.pp.filter_genes(eval(adata_s_name), min_counts=1)
    sc.pp.pca(eval(adata_s_name), random_state=seed, n_comps=30)
    tk.tl.triku(eval(adata_s_name), n_procs=1, random_state=seed)
    sc.pp.neighbors(eval(adata_s_name), metric='cosine')
    sc.tl.umap(eval(adata_s_name), min_dist=0.05, random_state=seed)

In [ ]:
fig, axs = plt.subplots(1, 2, figsize=(6, 3))
sc.pl.umap(adata_s1_13, color=['science_clustering'], legend_loc='on data', cmap=magma, ax=axs[0], show=False)
sc.pl.umap(adata_s1_2, color=['science_clustering'], legend_loc='on data', cmap=magma, ax=axs[1], show=False)

In [ ]:
fig, axs = plt.subplots(1, 2, figsize=(6, 3))
sc.pl.umap(adata_s2_13, color=['science_clustering'], legend_loc='on data', cmap=magma, ax=axs[0], show=False)
sc.pl.umap(adata_s2_2, color=['science_clustering'], legend_loc='on data', cmap=magma, ax=axs[1], show=False)

In [ ]:
fig, axs = plt.subplots(1, 2, figsize=(6, 3))
sc.pl.umap(adata_s3_13, color=['science_clustering'], legend_loc='on data', cmap=magma, ax=axs[0], show=False)
sc.pl.umap(adata_s3_2, color=['science_clustering'], legend_loc='on data', cmap=magma, ax=axs[1], show=False)

In [ ]:
fig, axs = plt.subplots(1, 2, figsize=(6, 3))
sc.pl.umap(adata_s4_13, color=['science_clustering'], legend_loc='on data', cmap=magma, ax=axs[0], show=False)
sc.pl.umap(adata_s4_2, color=['science_clustering'], legend_loc='on data', cmap=magma, ax=axs[1], show=False)

## Assigning JID categories
Based on this we do not see any discordancies in the dataset. We are going to apply the category assignation to assign the JID clusters to the dataset.

In [ ]:
## MAP JID populations

dict_cat = {'A1': ['PI16', 'QPCT', 'SLPI', 'CCN5', 'CPE', 'CTHRC1', 'MFAP5', 'PCOLCE2', 'SCARA5', 'TSPAN8'], 
            'A2': ['APCDD1', 'COL18A1', 'COMP', 'NKD2', 'F13A1', 'HSPB3', 'LEPR', 'TGFBI'], 
            'B1': ['CXCL2', 'MYC', 'C7', 'SPSB1', 'ITM2A'], #['CCL2', 'ITM2A', 'SPSB1', 'SOD2', 'APOD', 'C3', 'C7', 'CXCL2', 'CXCL12', 'DNAJA1', 'GGT5', 'IGFBP7', 'MYC'], 
            'B2': ['SOCS3', 'CCL19', 'CD74', 'RARRES2', 'CCDC146', 'IGFBP3', 'TNFSF13B'], 
            'C2': ['COCH', 'CRABP1', 'ARHGAP15', 'PLXDC1', 'RSPO4'], 
            'C3': ['ASPN', 'F2R', 'POSTN', 'TNN'],}

# dict_cat = {key: val[:4] for key, val in dict_cat.items()}

* A1: TSPAN8, SCARA5 less expressed in 1+3 datasets
* A2: F13A1, HSPB3 less expressed in 1+3 datasets. APCDD1 is not a good marker for all samples
* B1: removed many genes because expression patter is not consistent
* B2: CCDC146 + TNFSF13B not expressed in 1+3 datasets
* C1: Is not appearing
* C2: Is not appearing except in 2 datasets of some patients (s2 and S4)
* C3: Is not completely expressed, and sometimes POSTN does not collocalize with F2R
* C4: Is not present in the data

### S1 1+3

In [ ]:
adata_s = adata_s1_13

In [ ]:
for key, val in dict_cat.items():
        print(key, val)
        sc.pl.umap(adata_s, color=['science_clustering'] + [i for i in val[:4] if i in eval(adata_s_name).var_names], legend_loc='on data', cmap=magma, use_raw=False, ncols=5)

In [ ]:
sc.tl.leiden(adata_s, resolution=1.4, random_state=seed)
assign_cats(adata_s, dict_cats=dict_cat, column_groupby='leiden', intermediate_states=True, diff=0.05)
sc.pl.umap(adata_s, color=['leiden', 'assigned_cats', 
                           'assigned_cats_A1', 'assigned_cats_A2', 'assigned_cats_B1', 'assigned_cats_B2', 'assigned_cats_C2', 'assigned_cats_C3'], legend_loc='on data', vmax=1, vmin=0, cmap=magma, use_raw=False)

### S1 2

In [ ]:
adata_s = adata_s1_2

In [ ]:
for key, val in dict_cat.items():
        print(key, val)
        sc.pl.umap(adata_s, color=['science_clustering'] + [i for i in val if i in eval(adata_s_name).var_names], legend_loc='on data', cmap=magma, use_raw=False, ncols=5)

In [ ]:
sc.tl.leiden(adata_s, resolution=1.5, random_state=seed)
assign_cats(adata_s, dict_cats=dict_cat, column_groupby='leiden', intermediate_states=True, diff=0.07)
sc.pl.umap(adata_s, color=['leiden', 'assigned_cats', 
                           'assigned_cats_A1', 'assigned_cats_A2', 'assigned_cats_B1', 'assigned_cats_B2', 'assigned_cats_C2', 'assigned_cats_C3'], legend_loc='on data', vmax=1, cmap=magma, use_raw=False)

### S2 1+3

In [ ]:
adata_s = adata_s2_13

In [ ]:
for key, val in dict_cat.items():
        print(key, val)
        sc.pl.umap(adata_s, color=['science_clustering'] + [i for i in val if i in eval(adata_s_name).var_names], legend_loc='on data', cmap=magma, use_raw=False, ncols=5)

In [ ]:
sc.tl.leiden(adata_s, resolution=1.5, random_state=seed)
assign_cats(adata_s, dict_cats=dict_cat, column_groupby='leiden', intermediate_states=True, diff=0.07)
sc.pl.umap(adata_s, color=['leiden', 'assigned_cats', 
                           'assigned_cats_A1', 'assigned_cats_A2', 'assigned_cats_B1', 'assigned_cats_B2', 'assigned_cats_C2', 'assigned_cats_C3'], legend_loc='on data', vmax=1, cmap=magma, use_raw=False)

### S2 2

In [ ]:
adata_s = adata_s2_2

In [ ]:
for key, val in dict_cat.items():
        print(key, val)
        sc.pl.umap(adata_s, color=['science_clustering'] + [i for i in val if i in eval(adata_s_name).var_names], legend_loc='on data', cmap=magma, use_raw=False, ncols=5)

In [ ]:
sc.tl.leiden(adata_s, resolution=1, random_state=seed)
assign_cats(adata_s, dict_cats=dict_cat, column_groupby='leiden', intermediate_states=True, diff=0.07)
sc.pl.umap(adata_s, color=['leiden', 'assigned_cats', 
                           'assigned_cats_A1', 'assigned_cats_A2', 'assigned_cats_B1', 'assigned_cats_B2', 'assigned_cats_C2', 'assigned_cats_C3'], legend_loc='on data', vmax=1, cmap=magma, use_raw=False)

### S3 1+3

In [ ]:
adata_s = adata_s3_13

In [ ]:
for key, val in dict_cat.items():
        print(key, val)
        sc.pl.umap(adata_s, color=['science_clustering'] + [i for i in val if i in adata_s.var_names], legend_loc='on data', cmap=magma, use_raw=False, ncols=5)

In [ ]:
sc.tl.leiden(adata_s, resolution=2, random_state=seed)
assign_cats(adata_s, dict_cats=dict_cat, column_groupby='leiden', intermediate_states=True, diff=0.05)
sc.pl.umap(adata_s, color=['leiden', 'assigned_cats', 
                           'assigned_cats_A1', 'assigned_cats_A2', 'assigned_cats_B1', 'assigned_cats_B2', 'assigned_cats_C2', 'assigned_cats_C3'], legend_loc='on data', vmax=1, cmap=magma, use_raw=False)

### S3 2

In [ ]:
adata_s = adata_s3_2

In [ ]:
for key, val in dict_cat.items():
        print(key, val)
        sc.pl.umap(adata_s, color=['science_clustering'] + [i for i in val if i in adata_s.var_names], legend_loc='on data', cmap=magma, use_raw=False)

In [ ]:
sc.tl.leiden(adata_s, resolution=1.4, random_state=seed)
assign_cats(adata_s, dict_cats=dict_cat, column_groupby='leiden', intermediate_states=True, diff=0.07)
sc.pl.umap(adata_s, color=['leiden', 'assigned_cats', 
                           'assigned_cats_A1', 'assigned_cats_A2', 'assigned_cats_B1', 'assigned_cats_B2', 'assigned_cats_C2', 'assigned_cats_C3'], legend_loc='on data', vmax=1, cmap=magma, use_raw=False)

### S4 1+3

In [ ]:
adata_s = adata_s4_13

In [ ]:
for key, val in dict_cat.items():
        print(key, val)
        sc.pl.umap(adata_s, color=['science_clustering'] + [i for i in val if i in adata_s.var_names], legend_loc='on data', cmap=magma, use_raw=False)

In [ ]:
sc.tl.leiden(adata_s, resolution=2, random_state=seed)
assign_cats(adata_s, dict_cats=dict_cat, column_groupby='leiden', intermediate_states=True, diff=0.07)
sc.pl.umap(adata_s, color=['leiden', 'assigned_cats', 
                           'assigned_cats_A1', 'assigned_cats_A2', 'assigned_cats_B1', 'assigned_cats_B2', 'assigned_cats_C2', 'assigned_cats_C3'], legend_loc='on data', vmax=1, cmap=magma, use_raw=False)

### S4 2

In [ ]:
adata_s = adata_s4_2

In [ ]:
for key, val in dict_cat.items():
        print(key, val)
        sc.pl.umap(adata_s, color=['science_clustering'] + [i for i in val if i in adata_s.var_names], legend_loc='on data', cmap=magma, use_raw=False)

In [ ]:
sc.tl.leiden(adata_s, resolution=2, random_state=seed)
assign_cats(adata_s, dict_cats=dict_cat, column_groupby='leiden', intermediate_states=True, diff=0.07)
sc.pl.umap(adata_s, color=['leiden', 'assigned_cats', 
                           'assigned_cats_A1', 'assigned_cats_A2', 'assigned_cats_B1', 'assigned_cats_B2', 'assigned_cats_C2', 'assigned_cats_C3'], legend_loc='on data', vmax=1, cmap=magma, use_raw=False)

## Joining 1+3 and 2 adatas
Now we are going to join all adatas into two: one for each category. We should find that the predicted categories should match between them across adatas.

**OUTER JOIN IS APPLIED TO KEEPS GENES NOT EXPRESSED IN ALL ADATAS**

### No batch effect correction

In [ ]:
adata_all_13 = sc.AnnData.concatenate(adata_s1_13, adata_s2_13, adata_s3_13, adata_s4_13, batch_categories=['s1', 's2', 's3', 's4'], batch_key='s_dataset', join='outer')
adata_all_2 = sc.AnnData.concatenate(adata_s1_2, adata_s2_2, adata_s3_2, adata_s4_2, batch_categories=['s1', 's2', 's3', 's4'], batch_key='s_dataset', join='outer')

In [ ]:
for adata_name in ['adata_all_13', 'adata_all_2']:
    sc.pp.filter_genes(eval(adata_name), min_counts=1)
    sc.pp.pca(eval(adata_name), random_state=seed, n_comps=30)
    tk.tl.triku(eval(adata_name), n_procs=1, random_state=seed, use_adata_knn=True)
    sc.pp.neighbors(eval(adata_name), metric='cosine', random_state=seed)
    sc.tl.umap(eval(adata_name), min_dist=0.3, random_state=seed)

In [ ]:
for adata_name in ['adata_all_13', 'adata_all_2']:
    sc.pl.umap(eval(adata_name),  color=['s_dataset', 'assigned_cats'])

### bbknn

#### kNN within batch: 2

In [ ]:
adata_all_13 = sc.AnnData.concatenate(adata_s1_13, adata_s2_13, adata_s3_13, adata_s4_13, batch_categories=['s1', 's2', 's3', 's4'], batch_key='s_dataset', join='outer')
adata_all_2 = sc.AnnData.concatenate(adata_s1_2, adata_s2_2, adata_s3_2, adata_s4_2, batch_categories=['s1', 's2', 's3', 's4'], batch_key='s_dataset', join='outer')

In [ ]:
for adata_name in ['adata_all_13', 'adata_all_2']:
    sc.pp.filter_genes(eval(adata_name), min_counts=30)
    sc.pp.pca(eval(adata_name), random_state=seed, n_comps=30)
    sce.pp.bbknn(eval(adata_name), metric='angular', batch_key='s_dataset', neighbors_within_batch=2)
    tk.tl.triku(eval(adata_name), n_procs=1, random_state=seed, use_adata_knn=True)
    sc.tl.umap(eval(adata_name), min_dist=0.05, random_state=seed)

In [ ]:
for adata_name in ['adata_all_13', 'adata_all_2']:
    sc.tl.leiden(eval(adata_name), resolution=1.5)
    sc.pl.umap(eval(adata_name),  color=['s_dataset', 'leiden', 'assigned_cats'])

#### kNN within batch: 3

In [ ]:
adata_all_13 = sc.AnnData.concatenate(adata_s1_13, adata_s2_13, adata_s3_13, adata_s4_13, batch_categories=['s1', 's2', 's3', 's4'], batch_key='s_dataset', join='outer')
adata_all_2 = sc.AnnData.concatenate(adata_s1_2, adata_s2_2, adata_s3_2, adata_s4_2, batch_categories=['s1', 's2', 's3', 's4'], batch_key='s_dataset', join='outer')

In [ ]:
for adata_name in ['adata_all_13', 'adata_all_2']:
    sc.pp.filter_genes(eval(adata_name), min_counts=30)
    sc.pp.pca(eval(adata_name), random_state=seed, n_comps=30)
    sce.pp.bbknn(eval(adata_name), metric='angular', batch_key='s_dataset', neighbors_within_batch=3)
    tk.tl.triku(eval(adata_name), n_procs=1, random_state=seed, use_adata_knn=True)
    sc.tl.umap(eval(adata_name), min_dist=0.05, random_state=seed)

In [ ]:
for adata_name in ['adata_all_13', 'adata_all_2']:
    sc.tl.leiden(eval(adata_name), resolution=1.5)
    sc.pl.umap(eval(adata_name),  color=['s_dataset', 'leiden', 'assigned_cats'])

#### kNN within batch: 6

In [ ]:
adata_all_13 = sc.AnnData.concatenate(adata_s1_13, adata_s2_13, adata_s3_13, adata_s4_13, batch_categories=['s1', 's2', 's3', 's4'], batch_key='s_dataset', join='outer')
adata_all_2 = sc.AnnData.concatenate(adata_s1_2, adata_s2_2, adata_s3_2, adata_s4_2, batch_categories=['s1', 's2', 's3', 's4'], batch_key='s_dataset', join='outer')

In [ ]:
for adata_name in ['adata_all_13', 'adata_all_2']:
    sc.pp.filter_genes(eval(adata_name), min_counts=30)
    sc.pp.pca(eval(adata_name), random_state=seed, n_comps=30)
    sce.pp.bbknn(eval(adata_name), metric='angular', batch_key='s_dataset', neighbors_within_batch=6)
    tk.tl.triku(eval(adata_name), n_procs=1, random_state=seed, use_adata_knn=True)
    sc.tl.umap(eval(adata_name), min_dist=0.05, random_state=seed)

In [ ]:
for adata_name in ['adata_all_13', 'adata_all_2']:
    sc.pl.umap(eval(adata_name),  color=['s_dataset', 'assigned_cats'])

#### kNN within batch: 8

In [ ]:
adata_all_13 = sc.AnnData.concatenate(adata_s1_13, adata_s2_13, adata_s3_13, adata_s4_13, batch_categories=['s1', 's2', 's3', 's4'], batch_key='s_dataset', join='outer')
adata_all_2 = sc.AnnData.concatenate(adata_s1_2, adata_s2_2, adata_s3_2, adata_s4_2, batch_categories=['s1', 's2', 's3', 's4'], batch_key='s_dataset', join='outer')

In [ ]:
for adata_name in ['adata_all_13', 'adata_all_2']:
    sc.pp.filter_genes(eval(adata_name), min_counts=30)
    sc.pp.pca(eval(adata_name), random_state=seed, n_comps=30)
    sce.pp.bbknn(eval(adata_name), metric='angular', batch_key='s_dataset', neighbors_within_batch=8)
    tk.tl.triku(eval(adata_name), n_procs=1, random_state=seed, use_adata_knn=True)
    sc.tl.umap(eval(adata_name), min_dist=0.05, random_state=seed)

In [ ]:
for adata_name in ['adata_all_13', 'adata_all_2']:
    sc.pl.umap(eval(adata_name),  color=['s_dataset', 'assigned_cats'])

### harmonypy

#### sigma = 0.1

In [ ]:
adata_all_13 = sc.AnnData.concatenate(adata_s1_13, adata_s2_13, adata_s3_13, adata_s4_13, batch_categories=['s1', 's2', 's3', 's4'], batch_key='s_dataset', join='outer')
adata_all_2 = sc.AnnData.concatenate(adata_s1_2, adata_s2_2, adata_s3_2, adata_s4_2, batch_categories=['s1', 's2', 's3', 's4'], batch_key='s_dataset', join='outer')

In [ ]:
for adata_name in ['adata_all_13', 'adata_all_2']:
    sc.pp.filter_genes(eval(adata_name), min_counts=30)
    sc.pp.pca(eval(adata_name), random_state=seed, n_comps=30)
    sce.pp.harmony_integrate(eval(adata_name), key='s_dataset', basis='X_pca', 
                             adjusted_basis='X_pca_harmony', random_state=seed, 
                             epsilon_cluster=1e-06, epsilon_harmony=0.00001, 
                             max_iter_harmony=25, sigma=0.1)
    sc.pp.neighbors(eval(adata_name), metric='cosine', use_rep='X_pca_harmony', random_state=seed)
    tk.tl.triku(eval(adata_name), n_procs=1, random_state=seed, use_adata_knn=True)
    sc.tl.umap(eval(adata_name), min_dist=0.3, random_state=seed)

In [ ]:
for adata_name in ['adata_all_13', 'adata_all_2']:
    sc.tl.leiden(eval(adata_name), resolution=0.2)
    sc.tl.rank_genes_groups(eval(adata_name), method='wilcoxon', groupby='leiden')
    sc.pl.umap(eval(adata_name),  color=['s_dataset', 'leiden', 'assigned_cats',])

In [ ]:
sc.pl.umap(adata_all_13,  color=['s_dataset', 'leiden', 'assigned_cats',])

#### sigma = 0.25

In [ ]:
adata_all_13 = sc.AnnData.concatenate(adata_s1_13, adata_s2_13, adata_s3_13, adata_s4_13, batch_categories=['s1', 's2', 's3', 's4'], batch_key='s_dataset', join='outer')
adata_all_2 = sc.AnnData.concatenate(adata_s1_2, adata_s2_2, adata_s3_2, adata_s4_2, batch_categories=['s1', 's2', 's3', 's4'], batch_key='s_dataset', join='outer')

In [ ]:
for adata_name in ['adata_all_13', 'adata_all_2']:
    sc.pp.filter_genes(eval(adata_name), min_counts=30)
    sc.pp.pca(eval(adata_name), random_state=seed, n_comps=30)
    sce.pp.harmony_integrate(eval(adata_name), key='s_dataset', basis='X_pca', 
                             adjusted_basis='X_pca_harmony', random_state=seed, 
                             epsilon_cluster=1e-06, epsilon_harmony=0.00001, 
                             max_iter_harmony=25, sigma=0.25)
    sc.pp.neighbors(eval(adata_name), metric='cosine', use_rep='X_pca_harmony', random_state=seed)
    tk.tl.triku(eval(adata_name), n_procs=1, random_state=seed, use_adata_knn=True)
    sc.tl.umap(eval(adata_name), min_dist=0.3, random_state=seed)

In [ ]:
for adata_name in ['adata_all_13', 'adata_all_2']:
    sc.tl.leiden(eval(adata_name), resolution=0.6)
    sc.tl.rank_genes_groups(eval(adata_name), method='wilcoxon', groupby='leiden')
    sc.pl.umap(eval(adata_name),  color=['s_dataset', 'leiden', 'assigned_cats',])

In [ ]:
sc.pl.umap(adata_all_13,  color=['s_dataset', 'leiden', 'assigned_cats',])
sc.pl.rank_genes_groups_tracksplot(adata_all_13, dendrogram=False, use_raw=False, n_genes=20)

In [ ]:
sc.pl.umap(adata_all_2,  color=['s_dataset', 'leiden', 'assigned_cats',])
sc.pl.rank_genes_groups_tracksplot(adata_all_2, dendrogram=False, use_raw=False, n_genes=20)

#### sigma = 0.5

In [ ]:
adata_all_13 = sc.AnnData.concatenate(adata_s1_13, adata_s2_13, adata_s3_13, adata_s4_13, batch_categories=['s1', 's2', 's3', 's4'], batch_key='s_dataset', join='outer')
adata_all_2 = sc.AnnData.concatenate(adata_s1_2, adata_s2_2, adata_s3_2, adata_s4_2, batch_categories=['s1', 's2', 's3', 's4'], batch_key='s_dataset', join='outer')

In [ ]:
for adata_name in ['adata_all_13', 'adata_all_2']:
    sc.pp.filter_genes(eval(adata_name), min_counts=30)
    sc.pp.pca(eval(adata_name), random_state=seed, n_comps=30)
    sce.pp.harmony_integrate(eval(adata_name), key='s_dataset', basis='X_pca', 
                             adjusted_basis='X_pca_harmony', random_state=seed, 
                             epsilon_cluster=1e-06, epsilon_harmony=0.00001, 
                             max_iter_harmony=25, sigma=0.5)
    sc.pp.neighbors(eval(adata_name), metric='cosine', use_rep='X_pca_harmony', random_state=seed)
    tk.tl.triku(eval(adata_name), n_procs=1, random_state=seed, use_adata_knn=True)
    sc.tl.umap(eval(adata_name), min_dist=0.3, random_state=seed)

In [ ]:
for adata_name in ['adata_all_13', 'adata_all_2']:
    sc.tl.leiden(eval(adata_name), resolution=0.6)
    sc.tl.rank_genes_groups(eval(adata_name), method='wilcoxon', groupby='leiden')
    sc.pl.umap(eval(adata_name),  color=['s_dataset', 'leiden', 'assigned_cats',])

In [ ]:
sc.pl.umap(adata_all_13,  color=['s_dataset', 'leiden', 'assigned_cats',])
sc.pl.rank_genes_groups_tracksplot(adata_all_13, dendrogram=False, use_raw=False, n_genes=20)

In [ ]:
sc.pl.umap(adata_all_2,  color=['s_dataset', 'leiden', 'assigned_cats',])
sc.pl.rank_genes_groups_tracksplot(adata_all_2, dendrogram=False, use_raw=False, n_genes=20)

### Removing s4
We are going to try harmonypy with s=0.5 and bbknn with kNN within batch=2. The results we observe are similar to the ones obtained with s4, but we are more confident about the genes obtained.

#### harmonypy

In [ ]:
adata_all_13 = sc.AnnData.concatenate(adata_s1_13, adata_s2_13, adata_s3_13, batch_categories=['s1', 's2', 's3', 's4'], batch_key='s_dataset', join='outer')
adata_all_2 = sc.AnnData.concatenate(adata_s1_2, adata_s2_2, adata_s3_2, batch_categories=['s1', 's2', 's3', 's4'], batch_key='s_dataset', join='outer')

In [ ]:
for adata_name in ['adata_all_13', 'adata_all_2']:
    sc.pp.filter_genes(eval(adata_name), min_counts=30)
    sc.pp.pca(eval(adata_name), random_state=seed, n_comps=30)
    sce.pp.harmony_integrate(eval(adata_name), key='s_dataset', basis='X_pca', 
                             adjusted_basis='X_pca_harmony', random_state=seed, 
                             epsilon_cluster=1e-06, epsilon_harmony=0.00001, 
                             max_iter_harmony=25, sigma=0.5)
    sc.pp.neighbors(eval(adata_name), metric='cosine', use_rep='X_pca_harmony', random_state=seed)
    tk.tl.triku(eval(adata_name), n_procs=1, random_state=seed, use_adata_knn=True)
    sc.tl.umap(eval(adata_name), min_dist=0.3, random_state=seed)

In [ ]:
for adata_name in ['adata_all_13', 'adata_all_2']:
    sc.tl.leiden(eval(adata_name), resolution=0.6)
    sc.tl.rank_genes_groups(eval(adata_name), method='wilcoxon', groupby='leiden')
    sc.pl.umap(eval(adata_name),  color=['s_dataset', 'leiden', 'assigned_cats',])

#### bbknn

In [ ]:
adata_all_13 = sc.AnnData.concatenate(adata_s1_13, adata_s2_13, adata_s3_13, batch_categories=['s1', 's2', 's3'], batch_key='s_dataset', join='outer')
adata_all_2 = sc.AnnData.concatenate(adata_s1_2, adata_s2_2, adata_s3_2, batch_categories=['s1', 's2', 's3'], batch_key='s_dataset', join='outer')

In [ ]:
for adata_name in ['adata_all_13', 'adata_all_2']:
    sc.pp.filter_genes(eval(adata_name), min_counts=30)
    sc.pp.pca(eval(adata_name), random_state=seed, n_comps=30)
    sce.pp.bbknn(eval(adata_name), metric='angular', batch_key='s_dataset', neighbors_within_batch=2)
    tk.tl.triku(eval(adata_name), n_procs=1, random_state=seed, use_adata_knn=True)
    sc.tl.umap(eval(adata_name), min_dist=0.05, random_state=seed)

In [ ]:
for adata_name in ['adata_all_13', 'adata_all_2']:
    sc.tl.leiden(eval(adata_name), resolution=0.7)
    sc.pl.umap(eval(adata_name),  color=['s_dataset', 'leiden', 'assigned_cats'])

There are some interesting clusters that were appearing also with s4, and all the batch effect correction parameters.

In the datasets with 1+3 clusters we see that A2 cluster appears differentiated in 2/3 axis, both of them steming from the A1 cluster. One of the A2 clusters goes to the B1/B2 cluster (left, depending on the visualization) and the other one goes to the B2 cluster. We are going to study the DEGs to find differences in the two A2 clusters.

In the datasets with 2 cluster the clusters A1, A2 and B2 are clear. However, there is a 4th large cluster that has a mixture of cells. We are going to study that cluster as well.

In [ ]:
sc.pl.umap(adata_all_13,  color=['s_dataset', 'leiden', 'assigned_cats'])
sc.tl.rank_genes_groups(adata_all_13, groupby='leiden', method='wilcoxon')

In [ ]:
sc.pl.rank_genes_groups_tracksplot(adata_all_13, dendrogram=False, use_raw=False, n_genes=50)

In [ ]:
# THIS PART MIGHT NOT BE REPLICABLE OUTSIDE!!! Clusters 5 and 1 are set to detect differences in A2 left VS right
sc.tl.rank_genes_groups(adata_all_13, groupby='leiden', method='wilcoxon', groups=['5'], reference='1')
sc.pl.rank_genes_groups_tracksplot(adata_all_13, dendrogram=False, use_raw=False, n_genes=150)

In [ ]:
# THIS PART MIGHT NOT BE REPLICABLE OUTSIDE!!! Clusters 5 and 1 are set to detect differences in A2 left VS right
sc.tl.rank_genes_groups(adata_all_13, groupby='leiden', method='wilcoxon', groups=['1'], reference='5')
sc.pl.rank_genes_groups_tracksplot(adata_all_13, dendrogram=False, use_raw=False, n_genes=150)

In [ ]:
sc.pl.umap(adata_all_13, color=['leiden'] + list(adata_all_13.uns['rank_genes_groups']['names']['1'][:150]), cmap=magma, use_raw=False)

In [ ]:
genes_cluster_A2_L = ['HILPDA', 'VIM', 'FAM162A', 'VEGFA', 'LDHA', 'ADM', 'SNHG7', 'SYNPO', 'INSIG2', 'ERO1A', 'ZNF395', 'ANGPTL4', 
                      'NRN1', 'SLC2A1', 'EHD2', 'DDIT4', 'P4HA2', 'DPYSL2', 'WSB1', 'SNHG1', 'SNHG8', 'SLC16A3', 'BNIP3', 'RAB20', 'VKORC1',
                      'LOXL2', 'KLF6', 'PGM1', 'SLC2A14', 'PPP1R18', 'BAIAP2', 'CDON', 'CLK1', 'PDK1', 'PYGL', 'NFIX', 'AK4', 'RORA',
                      'NGLY1', 'SCD', 'GPRC5A', 'RASSF5', 'COPS2', 'COL27A1', 'SH3BP5', 'EGLN3', 'JUN', 'PGF', 'CXCR4', 'PPP1R3B', 
                      'HSD3B7', 'SNX33', 'KCTD11', 'NBFP9', 'MAFF', 'BEND5', 'SEPTIN9', 'PPP1R3C', 'SMIM3', 'SFXN3', 'SNHG18', 'OSBPL5',
                      ]
genes_cluster_A2_R = ['HSPE1', 'HMGB1', 'CEMIP', 'SNRPG', 'ERH', 'PRDX1', 'AKR1C1', 'PSMB3', 'DDX5', 'ATP5MF', 'CCT7', 'NDUFS6', 
                      'COX6B1', 'PSMA4', 'SPON1', 'NDUFAF3', 'ACTB', 'ADAMTS5', 'SLIRP', 'SSB', 'MRPL41', 'POLR2K', 'NQO2']

In [ ]:
assign_cats(adata_all_13, dict_cats={'L_A2_axis': genes_cluster_A2_L, 'R_A2_axis': genes_cluster_A2_R})

In [ ]:
dict_cat = {'fb': ['LUM', 'PDGFRA', 'COL1A1', 'SFRP2', 'CCL19'], 'peri': ['RGS5', 'MYL9', 'NDUFA4L2'], 'eritro': ['HBB', 'HBA2', 'HBA1'], 
            'immune': ['TRAC', 'HLA-DRA', 'FCER1G'], 'kerato': ['DMKN', 'KRT1'],
            'mt': ['MTND4P12', 'ADAM33', 'RN7SL2', ]}



We see that the expression of markers from A2_L is in fact scattered trough many clusters from the left axis. On the other hand, the genes from A2_R, although some of them are appearing in many clusters, is ore localized, and is expressed in either A or B axis. Also, there is a fewer difference between the two axes. This effect is much less apparent for genes in A2_L. To discriminate further a better list of genes, we are going to use these genes as markers of the *left* and *right* axis, and dilucidate better markers of the two axes.

In [ ]:
sc.pl.umap(adata_all_2,  color=['s_dataset', 'leiden', 'assigned_cats'])
sc.tl.rank_genes_groups(adata_all_2, groupby='leiden', method='wilcoxon')

In [ ]:
sc.pl.rank_genes_groups_tracksplot(adata_all_2, dendrogram=False, use_raw=False, n_genes=40)

Among all the options, we can choose bbknn with 2-3 neighbours, or harmonypy with sigma 0.5

In [ ]:
import gseapy as gp

In [ ]:
enr_13 = gp.enrichr(gene_list=list(genes_13),
                    gene_sets=['KEGG_2019_Human','GO_Biological_Process_2018'],
                 organism='Human', # don't forget to set organism to the one you desired! e.g. Yeast
                 description='test_name',
                 cutoff=0.5 # test dataset, use lower value from range(0,1)
                )

In [ ]:
selected_cols = ['Gene_set', 'Term', 'Adjusted P-value', 'Odds Ratio', 'Genes']

In [ ]:
enr_13.results.sort_values(by='Adjusted P-value').iloc[:15][selected_cols]

In [ ]:
enr_2 = gp.enrichr(gene_list=list(genes_2),
                    gene_sets=['KEGG_2019_Human','GO_Biological_Process_2018'],
                 organism='Human', # don't forget to set organism to the one you desired! e.g. Yeast
                 description='test_name',
                 cutoff=0.5 # test dataset, use lower value from range(0,1)
                )

In [ ]:
enr_2.results.sort_values(by='Adjusted P-value').iloc[:15][selected_cols]

In [ ]:
mitochondrial_genes = [i for i in adata_all.var_names if str(i).startswith('MT-')]

In [ ]:
sc.pl.umap(adata_all, color=['adata_s', 'MALAT1'] + mitochondrial_genes, legend_loc='on data', cmap=magma, use_raw=False)

In [ ]:
sc.pl.violin(adata_all, keys=mitochondrial_genes, use_raw=False, groupby='adata_s', n_cols=3)